In [1]:
#!/usr/bin/env python3
# -*- coding:utf-8 -*-import nltk
import nltk
from nltk.corpus import stopwords
from nltk.corpus import brown
from nltk.stem.porter import PorterStemmer
import numpy as np
import sys
import io

import urllib
from bs4 import BeautifulSoup

from nltk.corpus import wordnet as wn

import re
import os
import codecs
from sklearn import feature_extraction

from nltk import tokenize

import math

from nltk.tree import Tree
from nltk.parse.stanford import StanfordParser
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

import random

from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

from pythonrouge.pythonrouge import Pythonrouge

In [2]:
# read dirindex_path.txt file.

dir_index_path = {}
f_dirpath = open('dirindex_path.txt', 'r', encoding='utf-8')
for sents in f_dirpath:
    sents = sents.strip('\n')
    item1 = sents.split(':')
    dir_index_path[int(item1[0])] = item1[1]
f_dirpath.close()

In [4]:
# read docindex_path.txt file.

doc_index_path = {}
f_docpath = open('docindex_path.txt', 'r', encoding='utf-8')
for sents in f_docpath:
    sents = sents.strip('\n')
    item1 = sents.split(':')
    if int(item1[0]) not in doc_index_path:
        doc_index_path[int(item1[0])] = {}
        doc_index_path[int(item1[0])][int(item1[1])] = item1[2]
    else:
        doc_index_path[int(item1[0])][int(item1[1])] = item1[2]
f_docpath.close()

In [1]:
clufinal_ans_alldir = []
summarys_alldir = []

for dir_i in range(len(dir_index_path)):
    # each folder
    
    %run clustering-summarization-run.ipynb
    
    summarys_alldir.append(summarys)

In [33]:
path = 'evaluation_results/abstracts/phase1/SEEmodels/SEE.edited.abstracts.in.edus'
files = os.listdir(path)
reference_filedic = {}
symbols = {'[1]','[2]','[3]','[4]','[5]','[6]','[7]','[8]','[9]','[10]','[11]','[12]','[13]','[14]','[15]','[16]','[17]','[18]','[19]','[20]','[21]','[22]','[23]','[24]','[25]','[26]','[27]'}
for file in files:
    if file == '.DS_Store':
        continue
    file_list = file.split('.')
    file_list[0] = file_list[0][1:]
    one_path = path + "/" + file
    with open(one_path,'r',encoding='utf-8') as foo_file :
        soup = BeautifulSoup(foo_file)
        
    summary_fileone = []    
    
    f = []
    txt = soup.find_all('body')
    for item in txt:
        f.append(item.get_text())
        
    con = []
    for item in f:
        item = item.replace('\n',' ')
        for word in symbols:
            item = item.replace(word,' ')
        item_str = ''
        item_str = ' '.join(item.split())
        con.append(item_str)
    
    for j in range(len(con)):
        sents = nltk.sent_tokenize(con[j])
        for item0 in sents:
            summary_fileone.append(item0)
    
    if file_list[0] not in reference_filedic:
        reference_filedic[file_list[0]] = {}
        if file_list[1] == 'M':
            if file_list[2] not in reference_filedic[file_list[0]]:
                reference_filedic[file_list[0]][file_list[2]] = []
                reference_filedic[file_list[0]][file_list[2]].append(summary_fileone)
            else:
                reference_filedic[file_list[0]][file_list[2]].append(summary_fileone)
    else:
        if file_list[1] == 'M':
            if file_list[2] not in reference_filedic[file_list[0]]:
                reference_filedic[file_list[0]][file_list[2]] = []
                reference_filedic[file_list[0]][file_list[2]].append(summary_fileone)
            else:
                reference_filedic[file_list[0]][file_list[2]].append(summary_fileone)

In [35]:
reference_alldir = []
for dir_i in range(len(dir_index_path)):
    reference_alldir.append(reference_filedic[dir_index_path[dir_i][1:-1]]['100'])

In [2]:
# Output average Rouge scores of all folders.
    
rouge = Pythonrouge(summary_file_exist=False,
                    summary=summarys_alldir, reference=reference_alldir,
                    n_gram=2, ROUGE_SU4=True, ROUGE_L=True,
                    recall_only=True, stemming=True, stopwords=False,
                    word_level=True, length_limit=True, length=100,
                    use_cf=False, cf=95, scoring_formula='average',
                    resampling=True, samples=1000, favor=True, p=0.5)

score = rouge.calc_score()